In [1]:
from utils.dataLoader import DataLoader
from utils.modelLoader import ModelLoader
from utils.SiameseModel import SiameseModel
import tensorflow as tf
import pandas as pd

In [2]:
runs = {
    "Siamese_MobileNetV1_Train500": "metadate_identity-once_limit500.json",
    "Siamese_MobileNetV1_Train5000": "metadate_identity-once_limit5000.json",
    "Siamese_MobileNetV1_Train500_All": "metadate_all_limit500.json",
    "Siamese_MobileNetV1_Train5000_All": "metadate_all_limit5000.json"
}

In [3]:
for run in runs.keys():
    train_ds, val_ds, test_ds = DataLoader().loadDatasets("img_align_celeba", runs[run], 32)
    
    model, embedding = ModelLoader().loadMobileNetV1FaceRecognition(True)
    siamese_model = SiameseModel(siamese_network=model)
    siamese_model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), weighted_metrics=[])
    siamese_model.fit(train_ds, epochs=10, validation_data=val_ds)
    
    runPath = "../models/" + run
    model.save(runPath)
    embedding.save(runPath + "_embedding")

Epoch 1/10
10/11 [==========================>...] - ETA: 54s - loss: 0.4957 

In [2]:
train_ds, val_ds, test_ds = DataLoader().loadDatasets("img_align_celeba", "metadate_identity-once_limit500.json", 32)

In [3]:
model, embedding = ModelLoader().loadMobileNetV1FaceRecognition(True)
siamese_model = SiameseModel(siamese_network=model)
siamese_model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), weighted_metrics=[])
siamese_model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
11/11 [==============================] - 24s 413ms/step - loss: 0.4929 - val_loss: 0.4468
Epoch 2/10
11/11 [==============================] - 3s 256ms/step - loss: 0.3967 - val_loss: 0.3310
Epoch 3/10
11/11 [==============================] - 3s 249ms/step - loss: 0.2939 - val_loss: 0.2670
Epoch 4/10
11/11 [==============================] - 3s 252ms/step - loss: 0.2163 - val_loss: 0.2261
Epoch 5/10
11/11 [==============================] - 3s 250ms/step - loss: 0.1776 - val_loss: 0.1700
Epoch 6/10
11/11 [==============================] - 3s 250ms/step - loss: 0.1173 - val_loss: 0.1195
Epoch 7/10
 5/11 [============>.................] - ETA: 9:06 - loss: 0.1227 

In [ ]:
siamese_model.save("../models/Siamese_MobileNetV1_Train500")
embedding.save("../models/Siamese_MobileNetV1_Train500_embedding")

In [6]:
for sample in iter(test_ds):
    anchor, positive, negative = sample

    anchor_embedding, positive_embedding, negative_embedding = (
        embedding(tf.keras.applications.mobilenet.preprocess_input(anchor)),
        embedding(tf.keras.applications.mobilenet.preprocess_input(positive)),
        embedding(tf.keras.applications.mobilenet.preprocess_input(negative)),
    )
    
    cosine_similarity = tf.keras.metrics.CosineSimilarity()

    positive_similarity = cosine_similarity(anchor_embedding, positive_embedding)
    print("Positive similarity:", positive_similarity.numpy())

    negative_similarity = cosine_similarity(anchor_embedding, negative_embedding)
    print("Negative similarity", negative_similarity.numpy())
    
    print("Difference:                        ", positive_similarity.numpy()-negative_similarity.numpy())

Positive similarity: 0.78901875
Negative similarity 0.85778666
Difference:                         -0.068767905
Positive similarity: 0.951022
Negative similarity 0.86663395
Difference:                         0.08438808
Positive similarity: 0.88632905
Negative similarity 0.902148
Difference:                         -0.015818954
Positive similarity: 0.8191786
Negative similarity 0.8724378
Difference:                         -0.053259194
Positive similarity: 0.96817833
Negative similarity 0.9053608
Difference:                         0.062817514
Positive similarity: 0.83020896
Negative similarity 0.8059373
Difference:                         0.024271667
Positive similarity: 0.84479755
Negative similarity 0.85275865
Difference:                         -0.007961094
Positive similarity: 0.87677336
Negative similarity 0.86333275
Difference:                         0.013440609
Positive similarity: 0.91705555
Negative similarity 0.8657429
Difference:                         0.051312625
Positiv

def contrastive_loss(y, preds, margin=1):
    y = tf.cast(y, preds.dtype)
    squaredPreds = tf.keras.backend.square(preds)
    squaredMargin = tf.keras.backend.square(K.maximum(margin - preds, 0))
    loss = 1-tf.keras.backend.mean(y * squaredPreds + (1 - y) * squaredMargin) 
    return loss

data = pd.read_json("../data_meta/img_align_celeba/processed/metadate_pairs.json")
data_shuffled = data.sample(frac=1, random_state=123).reset_index(drop=True)
anchors = data_shuffled["anchor"].to_numpy()
compares = data_shuffled["compare"].to_numpy()
labels = data_shuffled["label"].to_numpy()
train_anchors = anchors[:int(len(anchors)*0.8)]
valid_anchors = anchors[int(len(anchors)*0.8):]
train_compares = compares[:int(len(anchors)*0.8)]
valid_compares = compares[int(len(anchors)*0.8):]
train_labels = labels[:int(len(anchors)*0.8)]
valid_labels = labels[int(len(anchors)*0.8):]

train_ds, val_ds, test_ds = DataLoader().loadDatasetsPairs("img_align_celeba", "metadate_pairs.json", 32)

model, embedding = ModelLoader().loadMobileNetV1FaceRecognitionPair()

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss=contrastive_loss, optimizer=opt, metrics=["accuracy"])

history = model.fit(train_ds, validation_data=val_ds, epochs=10)